In [ ]:
%cd ../src
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from loguru import logger
from datetime import datetime

import configuration
import evaluator

In [13]:
with open("../config_launch.json", "r", encoding="utf-8") as fin:
    config_launch = json.load(fin)
model_dicts = configuration.init_model_dicts()
if len(model_dicts) == 3:
    chatbot_model_dict, analyzer_model_dict, evaluator_model_dict = model_dicts
    use_provocator = False
else:
    use_provocator = True
    (
        chatbot_model_dict,
        analyzer_model_dict,
        evaluator_model_dict,
        provocator_model_dict,
    ) = model_dicts
    provocator_modes = [
        "hacker that wants to jailbreak the chatbot",
        "a person that types with a lot of typos and grammar mistakes",
        "a person who wants to make fun out of the chatbot",
    ]
logger.remove()

In [ ]:
factual_correctness_list = []
appropriateness_list = []
for i in range(1, 20):
    now = datetime.now()
    formatted_now = now.strftime("%Y-%m-%d_%H:%M:%S")
    logfile = f"../logs/{formatted_now}_{i}.txt"
    with open(logfile, "w") as fout:
        fout.write("evaluator_model_dict" + " " + str(evaluator_model_dict) + "\n")
        fout.write("chatbot_model_dict" + " " + str(chatbot_model_dict) + "\n")
        fout.write("analyzer_model_dict" + " " + str(analyzer_model_dict) + "\n")
        if use_provocator:
            fout.write(
                "provocator_model_dict" + " " + str(provocator_model_dict) + "\n"
            )

    logger.add(logfile)
    factual_correctness, appropriateness = evaluator.evaluate_scenario(
        i,
        model_dicts,
        config_launch["provocator_mode"],
    )
    factual_correctness_list.append(factual_correctness)
    appropriateness_list.append(appropriateness)
    logger.remove()

    with open(logfile, "a") as fout:
        print(
            factual_correctness_list,
            np.mean(np.array(factual_correctness_list)),
            np.std(np.array(factual_correctness_list)),
            file=fout,
        )
        print(
            appropriateness_list,
            np.mean(np.array(appropriateness_list)),
            np.std(np.array(appropriateness_list)),
            file=fout,
        )
        print("#" * 20, file=fout)